In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn python-dotenv python-multipart

In [2]:
!pip install pycaret

In [3]:
!pip install boto3

In [10]:
from pycaret.classification import *
import uvicorn
from fastapi import FastAPI, File, UploadFile, HTTPException
import pandas as pd
import os
import boto3


import dotenv

dotenv.load_dotenv("/content/drive/MyDrive/data/.env")
app = FastAPI()

In [11]:
class Model:
    def __init__(self, modelname, bucketname):
        self.model = load_model(modelname, platform = 'aws', authentication = { 'bucket' : bucketname })
    
    def predict(self, data):
        predictions = predict_model(self.model, data=data).Label.to_list()
        return predictions


In [12]:
model_et = Model("et_deployed", "lsmlops200020065")
model_lightgbm = Model("lightgbm_deployed", "lsmlops200020065")

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [14]:
@app.post("/et/predict")
async def create_upload_file(file: UploadFile = File(...)):
        if file.filename.endswith(".csv"):
        
            with open(file.filename, "wb")as f:
                f.write(file.file.read())
            data = pd.read_csv(file.filename)
            os.remove(file.filename)

            try:
                predictions = model_et.predict(data)
            except:
                raise HTTPException(status_code=406, detail="Invalid data format.")
            else:
                return {
                    "Labels": predictions
                }

        else:
            raise HTTPException(status_code=406, detail="Invalid file format. Only CSV Files accepted.")


@app.post(f"/lightgbm/predict")
async def create_upload_file(file: UploadFile = File(...)):
        if file.filename.endswith(".csv"):
        
            with open(file.filename, "wb")as f:
                f.write(file.file.read())
            data = pd.read_csv(file.filename)
            os.remove(file.filename)

            try:
                predictions = model_lightgbm.predict(data)
            except:
                raise HTTPException(status_code=406, detail="Invalid data format.")
            else:
                return {
                    "Labels": predictions
                }

        else:
            raise HTTPException(status_code=406, detail="Invalid file format. Only CSV Files accepted.")


if os.getenv("AWS_ACCESS_KEY_ID") == None or os.getenv("AWS_SECRET_ACCESS_KEY") == None:
    print("AWS Credentials missing. Please set required environment variables.")
    exit(1)

In [ ]:
import nest_asyncio
from pyngrok import ngrok

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://e39b20f567be.ngrok.io


INFO:     Started server process [1061]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     103.194.69.187:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     103.194.69.187:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     103.3.32.134:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     103.194.69.187:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     103.194.69.187:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     49.44.81.38:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     49.44.81.38:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     103.194.69.187:0 - "POST /et/predict HTTP/1.1" 200 OK
INFO:     103.194.69.187:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     103.194.69.187:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     103.194.69.187:0 - "POST /lightgbm/predict HTTP/1.1" 200 OK
